In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir kaggle

In [ ]:
!mv kaggle.json kaggle

In [ ]:
!chmod 600 /content/kaggle/kaggle.json

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/kaggle"

In [ ]:
!kaggle datasets download -d ikarus777/best-artworks-of-all-time

In [ ]:
!mkdir artworks && unzip -q best-artworks-of-all-time.zip -d artworks

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
ReduceLROnPlateau = tf.keras.callbacks.ReduceLROnPlateau
ModelCheckpoint = tf.keras.callbacks.ModelCheckpoint
load_model = tf.keras.models.load_model

load_img = tf.keras.preprocessing.image.load_img
img_to_array = tf.keras.preprocessing.image.img_to_array
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator

Adam = tf.keras.optimizers.Adam
import pandas as pd

In [ ]:
import plotly
import plotly.graph_objs as go
from plotly import tools

# **การทดลอง S3**

In [ ]:
vgg16_model = tf.keras.applications.VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

In [ ]:
vgg16_model.trainable = False

In [ ]:
vgg16_model.summary()

In [ ]:
vgg16_model = tf.keras.models.Model(inputs=vgg16_model.inputs, outputs=vgg16_model.layers[-2].output)
vgg16_model.summary()

In [ ]:
output = tf.keras.layers.Dense(11, activation='softmax')(vgg16_model.layers[-1].output)
model = tf.keras.Model(inputs=vgg16_model.inputs, outputs=output)

In [ ]:
adam_optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])

model.summary()

In [ ]:
artists_df = pd.read_csv('artworks/artists.csv')
artists_df.head()

In [ ]:
artists_df.sort_values(by=['paintings'], ascending=False, inplace=True)
artists_df.head(15)

In [ ]:
artists_top = artists_df[artists_df['paintings'] >= 200].reset_index()
artists_top.shape

In [ ]:
artists_top = artists_top[['name', 'paintings']]
artists_top.head()

In [ ]:
artists_top['class_weight'] = artists_top.paintings.sum()/(artists_top.shape[0] * artists_top.paintings)
artists_top

In [ ]:
class_weights = artists_top['class_weight'].to_dict()
class_weights

In [ ]:
artists_top['name']

In [ ]:
artists_top_name = artists_top['name'].str.replace(' ', '_')
artists_top_name

In [ ]:
artists_class = artists_top_name.tolist()
artists_class

In [ ]:
datagen = ImageDataGenerator(validation_split=0.2,
                             preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
                             horizontal_flip=True)

valid_datagen=ImageDataGenerator(validation_split=0.2,
                                 preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

In [ ]:
train_artworks = datagen.flow_from_directory('artworks/images/images',
                                      subset='training',
                                      class_mode='categorical',
                                      target_size=(224, 224),
                                      color_mode='rgb',
                                      batch_size=64,
                                      shuffle=False,
                                      seed=99,
                                      classes=artists_class)

In [ ]:
batch_x, batch_y = train_artworks.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batch_x.shape, batch_x.min(), batch_x.max()))

In [ ]:
validate_artworks = valid_datagen.flow_from_directory('artworks/images/images',
                                      subset='validation',
                                      class_mode='categorical',
                                      target_size=(224, 224),
                                      color_mode='rgb',
                                      batch_size=64,
                                      shuffle=False,
                                      seed=99,
                                      classes=artists_class)

In [ ]:
batch_x, batch_y = validate_artworks.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batch_x.shape, batch_x.min(), batch_x.max()))

In [ ]:
batch_y.shape

In [ ]:
SPE_TRAIN = train_artworks.n//train_artworks.batch_size
SPE_VALID = validate_artworks.n//validate_artworks.batch_size
SPE_TRAIN, SPE_VALID

In [ ]:
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)

filepath="weights_best_s3.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint, rlrp]

In [ ]:
history = model.fit(train_artworks,
                    validation_data=validate_artworks,
                    epochs=20,
                    steps_per_epoch= SPE_TRAIN,
                    validation_steps=SPE_VALID,
                    shuffle=True,
                    verbose = 1,
                    class_weight=class_weights,
                    callbacks=callbacks_list)

In [ ]:
h1 = go.Scatter(y=history.history['loss'], 
                    mode="lines",
                    line=dict(
                      width=2,
                      color='blue'),
                      name="loss"
                   )
h2 = go.Scatter(y=history.history['val_loss'], 
                    mode="lines",
                    line=dict(
                      width=2,
                      color='red'),
                      name="val_loss"
                   )

data = [h1, h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
fig1.show()

In [ ]:
h1 = go.Scatter(y=history.history['accuracy'], 
                    mode="lines",
                    line=dict(
                      width=2,
                      color='blue'),
                      name="loss"
                   )
h2 = go.Scatter(y=history.history['val_accuracy'], 
                    mode="lines",
                    line=dict(
                      width=2,
                      color='red'),
                      name="val_loss"
                   )

data = [h1, h2]
layout1 = go.Layout(title='Accuracy',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
fig1.show()

In [ ]:
model = load_model(filepath)
score = model.evaluate(validate_artworks, verbose=1)

# **การทดลอง S4**

In [ ]:
vgg16_model = tf.keras.applications.VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

In [ ]:
vgg16_model.trainable = False

In [ ]:
vgg16_model = tf.keras.Model(inputs=vgg16_model.inputs, outputs=vgg16_model.layers[-2].output)

In [ ]:
vgg16_model.get_layer('fc2').trainable = True
vgg16_model.get_layer('fc1').trainable = True

In [ ]:
output = tf.keras.layers.Dense(11, activation='softmax')(vgg16_model.layers[-1].output)
model = tf.keras.Model(inputs=vgg16_model.inputs, outputs=output)

In [ ]:
adam_optimizer = Adam(learning_rate=0.00001)

model.compile(loss='categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])
model.summary()

In [ ]:
filepath="weights_best_s4.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint, rlrp]

In [ ]:
history2 = model.fit(train_artworks,
                    validation_data=validate_artworks,
                    epochs=20,
                    steps_per_epoch= SPE_TRAIN,
                    validation_steps=SPE_VALID,
                    shuffle=True,
                    verbose = 1,
                    class_weight=class_weights,
                    callbacks=callbacks_list)

In [ ]:
h1 = go.Scatter(y=history2.history['loss'], 
                    mode="lines",
                    line=dict(
                      width=2,
                      color='blue'),
                      name="loss"
                   )
h2 = go.Scatter(y=history2.history['val_loss'], 
                    mode="lines",
                    line=dict(
                      width=2,
                      color='red'),
                      name="val_loss"
                   )

data = [h1, h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
fig1.show()

In [ ]:
h1 = go.Scatter(y=history2.history['accuracy'], 
                    mode="lines",
                    line=dict(
                      width=2,
                      color='blue'),
                      name="loss"
                   )
h2 = go.Scatter(y=history2.history['val_accuracy'], 
                    mode="lines",
                    line=dict(
                      width=2,
                      color='red'),
                      name="val_loss"
                   )

data = [h1, h2]
layout1 = go.Layout(title='Accuracy',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
fig1.show()

In [ ]:
model = load_model(filepath)
score = model.evaluate(validate_artworks, verbose=1)

# **การทดลอง S5**

In [ ]:
vgg16_model = tf.keras.applications.VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

In [ ]:
vgg16_model.trainable = False

In [ ]:
vgg16_model = tf.keras.Model(inputs=vgg16_model.inputs, outputs=vgg16_model.layers[-2].output)

In [ ]:
output = tf.keras.layers.Dense(11, activation='softmax')(vgg16_model.layers[-1].output)
model = tf.keras.Model(inputs=vgg16_model.inputs, outputs=output)

In [ ]:
for layer in model.layers[10:]:
    layer.trainable = True

In [ ]:
adam_optimizer = Adam(learning_rate=0.00001)

model.compile(loss='categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])
model.summary()

In [ ]:
filepath="weights_best_s5.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint, rlrp]

In [ ]:
history3 = model.fit(train_artworks,
                    validation_data=validate_artworks,
                    epochs=20,
                    steps_per_epoch= SPE_TRAIN,
                    validation_steps=SPE_VALID,
                    shuffle=True,
                    verbose = 1,
                    class_weight=class_weights,
                    callbacks=callbacks_list)

In [ ]:
h1 = go.Scatter(y=history3.history['loss'], 
                    mode="lines",
                    line=dict(
                      width=2,
                      color='blue'),
                      name="loss"
                   )
h2 = go.Scatter(y=history3.history['val_loss'], 
                    mode="lines",
                      line=dict(
                      width=2,
                      color='red'),
                      name="val_loss"
                   )

data = [h1, h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
fig1.show()

In [ ]:
h1 = go.Scatter(y=history3.history['accuracy'], 
                    mode="lines",
                    line=dict(
                      width=2,
                      color='blue'),
                      name="loss"
                   )
h2 = go.Scatter(y=history3.history['val_accuracy'], 
                    mode="lines",
                    line=dict(
                    width=2,
                    color='red'),
                    name="val_loss"
                   )

data = [h1, h2]
layout1 = go.Layout(title='Accuracy',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
fig1.show()

In [ ]:
model = load_model(filepath)
score = model.evaluate(validate_artworks, verbose=1)

# **การทดลอง S6**

In [ ]:
vgg16_model = tf.keras.applications.VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

In [ ]:
vgg16_model.trainable = True

In [ ]:
vgg16_model = tf.keras.Model(inputs=vgg16_model.inputs, outputs=vgg16_model.layers[-2].output)

In [ ]:
output = tf.keras.layers.Dense(11, activation='softmax')(vgg16_model.layers[-1].output)
model = tf.keras.Model(inputs=vgg16_model.inputs, outputs=output)

In [ ]:
adam_optimizer = Adam(learning_rate=0.00001)

model.compile(loss='categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])
model.summary()

In [ ]:
filepath="weights_best_s6.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint, rlrp]

In [ ]:
history4 = model.fit(train_artworks,
                    validation_data=validate_artworks,
                    epochs=20,
                    steps_per_epoch= SPE_TRAIN,
                    validation_steps=SPE_VALID,
                    shuffle=True,
                    verbose = 1,
                    class_weight=class_weights,
                    callbacks=callbacks_list)

In [ ]:
h1 = go.Scatter(y=history4.history['loss'], 
                    mode="lines",
                    line=dict(
                      width=2,
                      color='blue'),
                      name="loss"
                   )
h2 = go.Scatter(y=history4.history['val_loss'], 
                    mode="lines",
                    line=dict(
                      width=2,
                      color='red'),
                      name="val_loss"
                   )

data = [h1, h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
fig1.show()

In [ ]:
h1 = go.Scatter(y=history4.history['accuracy'], 
                    mode="lines",
                    line=dict(
                      width=2,
                      color='blue'),
                      name="loss"
                   )
h2 = go.Scatter(y=history4.history['val_accuracy'], 
                    mode="lines",
                    line=dict(
                      width=2,
                      color='red'),
                      name="val_loss"
                   )

data = [h1, h2]
layout1 = go.Layout(title='Accuracy',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
fig1.show()

In [ ]:
model = load_model(filepath)
score = model.evaluate(validate_artworks, verbose=1)